In [1]:
import numpy as np
import pandas as pd
import scipy
from datetime import datetime
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

%matplotlib inline
warnings.filterwarnings('ignore')
sns.set_style('whitegrid')
sns.set_palette('muted')

## Load Data

In [2]:
eth_raw = pd.read_csv('eth_raw.csv')
print(eth_raw.info())
eth_raw.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1131 entries, 0 to 1130
Data columns (total 7 columns):
close         1131 non-null float64
high          1131 non-null float64
low           1131 non-null float64
open          1131 non-null float64
volumefrom    1131 non-null float64
volumeto      1131 non-null float64
timestamp     1131 non-null object
dtypes: float64(6), object(1)
memory usage: 61.9+ KB
None


,close,high,low,open,volumefrom,volumeto,timestamp
0,3.00,3.0,0.6747,0.6747,123.93,371.79,2015-08-06 19:00:00
1,1.20,3.0,0.1500,3.0000,2119.43,1438.16,2015-08-07 19:00:00
2,1.20,1.2,1.2000,1.2000,0.00,0.00,2015-08-08 19:00:00
3,1.20,1.2,1.2000,1.2000,0.00,0.00,2015-08-09 19:00:00
4,0.99,1.2,0.6504,1.2000,9486.09,7419.73,2015-08-10 19:00:00


In [3]:
eth = eth_raw[['timestamp', 'open', 'high', 'low', 'close', 'volumefrom', 'volumeto']].copy()
eth['timestamp'] = pd.to_datetime(eth['timestamp'])
eth['timestamp'] = eth['timestamp'].dt.date
eth.rename(columns = {'timestamp':'date'}, inplace=True)
eth['date'] = pd.to_datetime(eth['date'], format='%Y-%m-%d')
eth.head(2)

,date,open,high,low,close,volumefrom,volumeto
0,2015-08-06,0.6747,3.0,0.6747,3.0,123.93,371.79
1,2015-08-07,3.0000,3.0,0.1500,1.2,2119.43,1438.16


In [4]:
eth.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1131 entries, 0 to 1130
Data columns (total 7 columns):
date          1131 non-null datetime64[ns]
open          1131 non-null float64
high          1131 non-null float64
low           1131 non-null float64
close         1131 non-null float64
volumefrom    1131 non-null float64
volumeto      1131 non-null float64
dtypes: datetime64[ns](1), float64(6)
memory usage: 61.9 KB


In [48]:
# save df
eth.to_pickle('eth_df.pkl')

## Exogenous Variables
- Etherem related
    - high
    - low
    - open
    - volume
    - market cap
- Other Cryptocurrencies
    - BTC: Bitcoin
    - XRP: Ripple
    - EOS: EOS.IO
    - LTC: Litecoin
    - XLM: Stellar
    - XMR: Monero
- Other Indices
    - VIX
    - DXY
    - EFFR

In [ ]:
#eth_market_cap = pd.read_csv('ethereum_price.csv')
#print(eth_market_cap.info())
#eth_market_cap.head(2)

In [ ]:
#eth_market_cap['Date'] = pd.to_datetime(eth_market_cap['Date'],
#                                        format='%b %d, %Y')
#eth_market_cap.sort_values('Date', inplace=True)
#eth_market_cap.reset_index(drop=True, inplace=True)
#eth_market_cap['Market Cap'] = pd.to_numeric(
#    eth_market_cap['Market Cap'].str.replace(',', ''),
#    errors='coerce')
#eth_market_cap.rename(columns={'Date':'date', 'Market Cap':'market_cap'}, inplace=True)
#eth_market_cap.head(2)

In [ ]:
#eth_market_cap.drop(['Open', 'High', 'Low', 'Close', 'Volume'], axis=1, inplace=True)

In [ ]:
#df = eth.merge(eth_market_cap, how='outer', on='date')

In [ ]:
#df.head(2)

### **[VIX: CBOE Volatility Index](https://en.wikipedia.org/wiki/VIX)**
- Measure of stock market's expectation of volatility implied by S&P 500 index options, aka 'fear index'
- [Data Source](http://www.cboe.com/products/vix-index-volatility/vix-options-and-futures/vix-index/vix-historical-data)

In [108]:
df = eth.copy()

In [109]:
vix_raw = pd.read_csv('vixcurrent.csv', header=1)
print(vix_raw.info())
vix_raw[:1]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3697 entries, 0 to 3696
Data columns (total 5 columns):
Date         3697 non-null object
VIX Open     3697 non-null float64
VIX High     3697 non-null float64
VIX Low      3697 non-null float64
VIX Close    3697 non-null float64
dtypes: float64(4), object(1)
memory usage: 144.5+ KB
None


,Date,VIX Open,VIX High,VIX Low,VIX Close
0,1/2/2004,17.96,18.68,17.54,18.22


In [110]:
start_date = '2015-08-06'
end_date = '2018-09-09'
vix_close = vix_raw[['Date', 'VIX Close']].copy()
vix_close['Date'] = pd.to_datetime(vix_close['Date'], format='%m/%d/%Y')
vix_close = vix_close.loc[(vix_close['Date'] >= start_date)
                          & (vix_close['Date'] <= end_date)]
vix_close.sort_values('Date', inplace=True)
vix_close.reset_index(drop=True, inplace=True)
vix_close.columns = ['date', 'vix']
vix_close.head()

,date,vix
0,2015-08-06,13.77
1,2015-08-07,13.39
2,2015-08-10,12.23
3,2015-08-11,13.71
4,2015-08-12,13.61


In [111]:
df = df.merge(vix_close, how='outer', on='date')
df.vix.ffill(inplace=True)
#df_04ex.vix[0]= vix_raw.loc[vix_raw['Date'] == '11/24/2017', ['VIX Close']].values
print(df.vix.isnull().sum())
df.head()

0


,date,open,high,low,close,volumefrom,volumeto,vix
0,2015-08-06,0.6747,3.0,0.6747,3.00,123.93,371.79,13.77
1,2015-08-07,3.0000,3.0,0.1500,1.20,2119.43,1438.16,13.39
2,2015-08-08,1.2000,1.2,1.2000,1.20,0.00,0.00,13.39
3,2015-08-09,1.2000,1.2,1.2000,1.20,0.00,0.00,13.39
4,2015-08-10,1.2000,1.2,0.6504,0.99,9486.09,7419.73,12.23


### **[DXY: US Dollar Index](https://en.wikipedia.org/wiki/U.S._Dollar_Index)**
- Measure of value of USD relative to a basket of foreign currencies
- [Data Source](https://quotes.wsj.com/index/DXY/historical-prices)

In [112]:
dxy_raw = pd.read_csv('dxy_historical.csv')
print(dxy_raw.info())
dxy_raw[:1]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 658 entries, 0 to 657
Data columns (total 5 columns):
Date      658 non-null object
 Open     658 non-null float64
 High     658 non-null float64
 Low      658 non-null float64
 Close    658 non-null float64
dtypes: float64(4), object(1)
memory usage: 25.8+ KB
None


,Date,Open,High,Low,Close
0,02/20/18,89.24,89.8,89.22,89.72


In [113]:
dxy = dxy_raw[['Date', ' Close']].copy()
dxy['Date'] = pd.to_datetime(dxy['Date'], format='%x')
dxy = dxy.loc[(dxy['Date'] >= start_date)
              & (dxy['Date'] <= end_date)]
dxy.sort_values('Date', inplace=True)
dxy.reset_index(drop=True, inplace=True)
dxy.columns = ['date', 'dxy']
dxy.head()

,date,dxy
0,2015-08-07,97.56
1,2015-08-10,97.19
2,2015-08-11,97.20
3,2015-08-12,96.29
4,2015-08-13,96.36


In [114]:
df = df.merge(dxy, how='outer', on='date')
df.ffill(inplace=True)

# get first value
#dxy_raw['Date'] = pd.to_datetime(dxy_raw['Date'], format='%x')
#df_04ex.dxy[0] = dxy_raw.loc[dxy_raw['Date'] == '11/24/2017', [' Close']].values
print(df.dxy.isnull().sum())
df.head(2)

1


,date,open,high,low,close,volumefrom,volumeto,vix,dxy
0,2015-08-06,0.6747,3.0,0.6747,3.0,123.93,371.79,13.77,NaN
1,2015-08-07,3.0000,3.0,0.1500,1.2,2119.43,1438.16,13.39,97.56


In [115]:
df.tail(2)

,date,open,high,low,close,volumefrom,volumeto,vix,dxy
1129,2018-09-08,196.77,208.38,185.01,195.99,803730.43,1.580799e+08,14.88,89.72
1130,2018-09-09,195.99,202.59,186.92,196.98,649556.50,1.268624e+08,14.88,89.72


### **[EFFR: Federal Funds Rate](https://en.wikipedia.org/wiki/Federal_funds_rate)**
- The interest rate at which depository institutions (banks and credit unions) lend reserve balances to other depository institutions overnight, on an uncollateralized basis 
- [Data Source](https://fred.stlouisfed.org/series/EFFR)

In [116]:
effr_raw = pd.read_csv('EFFR.csv')
effr_raw.head()

,DATE,EFFR
0,2013-09-03,0.09
1,2013-09-04,0.08
2,2013-09-05,0.08
3,2013-09-06,0.08
4,2013-09-09,0.08


In [117]:
effr = effr_raw.copy()
effr.columns = ['date', 'effr']
effr['date'] = pd.to_datetime(effr['date'], format='%Y-%m-%d')
effr.sort_values('date', inplace=True)
effr = effr.loc[(effr['date'] >= start_date) & (effr['date'] <= end_date)]
effr.reset_index(drop=True, inplace=True)
effr.head()

,date,effr
0,2015-08-06,0.14
1,2015-08-07,0.14
2,2015-08-10,0.14
3,2015-08-11,0.15
4,2015-08-12,0.15


In [118]:
df = df.merge(effr, how='outer', on='date')
df.ffill(inplace=True)
print(df.effr.isnull().sum())
df.head()

0


,date,open,high,low,close,volumefrom,volumeto,vix,dxy,effr
0,2015-08-06,0.6747,3.0,0.6747,3.00,123.93,371.79,13.77,NaN,0.14
1,2015-08-07,3.0000,3.0,0.1500,1.20,2119.43,1438.16,13.39,97.56,0.14
2,2015-08-08,1.2000,1.2,1.2000,1.20,0.00,0.00,13.39,97.56,0.14
3,2015-08-09,1.2000,1.2,1.2000,1.20,0.00,0.00,13.39,97.56,0.14
4,2015-08-10,1.2000,1.2,0.6504,0.99,9486.09,7419.73,12.23,97.19,0.14


## Add other coins

In [119]:
# bitcoin
btc_raw = pd.read_csv('btc_raw.csv')
btc = btc_raw[['timestamp', 'open', 'high', 'low', 'close', 'volumefrom', 'volumeto']].copy()
btc['timestamp'] = pd.to_datetime(btc['timestamp'])
btc['timestamp'] = btc['timestamp'].dt.date
btc.rename(columns = {'timestamp':'date', 'close':'btc'}, inplace=True)
btc['date'] = pd.to_datetime(btc['date'], format='%Y-%m-%d')
btc = btc.loc[(btc['date'] >= start_date) & (btc['date'] <= end_date)]
btc.sort_values(by='date', inplace=True)
btc.reset_index(drop=True, inplace=True)
btc.drop(['high', 'low', 'open', 'volumefrom', 'volumeto'], axis=1, inplace=True)
btc.head(2)

,date,btc
0,2015-08-06,277.89
1,2015-08-07,258.60


In [120]:
df = df.merge(btc, how='outer', on='date')
print(df.shape)
df.head(3)

(1131, 11)


,date,open,high,low,close,volumefrom,volumeto,vix,dxy,effr,btc
0,2015-08-06,0.6747,3.0,0.6747,3.0,123.93,371.79,13.77,NaN,0.14,277.89
1,2015-08-07,3.0000,3.0,0.1500,1.2,2119.43,1438.16,13.39,97.56,0.14,258.60
2,2015-08-08,1.2000,1.2,1.2000,1.2,0.00,0.00,13.39,97.56,0.14,263.87


In [121]:
# ripple
xrp_raw = pd.read_csv('xrp_raw.csv')
xrp = xrp_raw[['timestamp', 'open', 'high', 'low', 'close', 'volumefrom', 'volumeto']].copy()
xrp['timestamp'] = pd.to_datetime(xrp['timestamp'])
xrp['timestamp'] = xrp['timestamp'].dt.date
xrp.rename(columns = {'timestamp':'date', 'close':'xrp'}, inplace=True)
xrp['date'] = pd.to_datetime(xrp['date'], format='%Y-%m-%d')
xrp = xrp.loc[(xrp['date'] >= start_date) & (xrp['date'] <= end_date)]
xrp.sort_values(by='date', inplace=True)
xrp.reset_index(drop=True, inplace=True)
xrp.drop(['high', 'low', 'open', 'volumefrom', 'volumeto'], axis=1, inplace=True)
df = df.merge(xrp, how='outer', on='date')
print(df.shape)
df.head(3)

(1131, 12)


,date,open,high,low,close,volumefrom,volumeto,vix,dxy,effr,btc,xrp
0,2015-08-06,0.6747,3.0,0.6747,3.0,123.93,371.79,13.77,NaN,0.14,277.89,0.008013
1,2015-08-07,3.0000,3.0,0.1500,1.2,2119.43,1438.16,13.39,97.56,0.14,258.60,0.009000
2,2015-08-08,1.2000,1.2,1.2000,1.2,0.00,0.00,13.39,97.56,0.14,263.87,0.008700


In [122]:
# eos.io
eos_raw = pd.read_csv('eos_raw.csv')
eos = eos_raw[['timestamp', 'open', 'high', 'low', 'close', 'volumefrom', 'volumeto']].copy()
eos['timestamp'] = pd.to_datetime(eos['timestamp'])
eos['timestamp'] = eos['timestamp'].dt.date
eos.rename(columns = {'timestamp':'date', 'close':'eos'}, inplace=True)
eos['date'] = pd.to_datetime(eos['date'], format='%Y-%m-%d')
eos = eos.loc[(eos['date'] >= start_date) & (eos['date'] <= end_date)]
eos.sort_values(by='date', inplace=True)
eos.reset_index(drop=True, inplace=True)
eos.drop(['high', 'low', 'open', 'volumefrom', 'volumeto'], axis=1, inplace=True)
df = df.merge(eos, how='outer', on='date')
print(df.shape)
df.head(3)

(1131, 13)


,date,open,high,low,close,volumefrom,volumeto,vix,dxy,effr,btc,xrp,eos
0,2015-08-06,0.6747,3.0,0.6747,3.0,123.93,371.79,13.77,NaN,0.14,277.89,0.008013,NaN
1,2015-08-07,3.0000,3.0,0.1500,1.2,2119.43,1438.16,13.39,97.56,0.14,258.60,0.009000,NaN
2,2015-08-08,1.2000,1.2,1.2000,1.2,0.00,0.00,13.39,97.56,0.14,263.87,0.008700,NaN


In [123]:
# litecoin
ltc_raw = pd.read_csv('ltc_raw.csv')
ltc = ltc_raw[['timestamp', 'open', 'high', 'low', 'close', 'volumefrom', 'volumeto']].copy()
ltc['timestamp'] = pd.to_datetime(ltc['timestamp'])
ltc['timestamp'] = ltc['timestamp'].dt.date
ltc.rename(columns = {'timestamp':'date', 'close':'ltc'}, inplace=True)
ltc['date'] = pd.to_datetime(ltc['date'], format='%Y-%m-%d')
ltc = ltc.loc[(ltc['date'] >= start_date) & (ltc['date'] <= end_date)]
ltc.sort_values(by='date', inplace=True)
ltc.reset_index(drop=True, inplace=True)
ltc.drop(['high', 'low', 'open', 'volumefrom', 'volumeto'], axis=1, inplace=True)
df = df.merge(ltc, how='outer', on='date')
print(df.shape)
df.head(3)

(1131, 14)


,date,open,high,low,close,volumefrom,volumeto,vix,dxy,effr,btc,xrp,eos,ltc
0,2015-08-06,0.6747,3.0,0.6747,3.0,123.93,371.79,13.77,NaN,0.14,277.89,0.008013,NaN,4.10
1,2015-08-07,3.0000,3.0,0.1500,1.2,2119.43,1438.16,13.39,97.56,0.14,258.60,0.009000,NaN,3.77
2,2015-08-08,1.2000,1.2,1.2000,1.2,0.00,0.00,13.39,97.56,0.14,263.87,0.008700,NaN,3.87


In [124]:
# stellar
xlm_raw = pd.read_csv('xlm_raw.csv')
xlm = xlm_raw[['timestamp', 'open', 'high', 'low', 'close', 'volumefrom', 'volumeto']].copy()
xlm['timestamp'] = pd.to_datetime(xlm['timestamp'])
xlm['timestamp'] = xlm['timestamp'].dt.date
xlm.rename(columns = {'timestamp':'date', 'close':'xlm'}, inplace=True)
xlm['date'] = pd.to_datetime(xlm['date'], format='%Y-%m-%d')
xlm = xlm.loc[(xlm['date'] >= start_date) & (xlm['date'] <= end_date)]
xlm.sort_values(by='date', inplace=True)
xlm.reset_index(drop=True, inplace=True)
xlm.drop(['high', 'low', 'open', 'volumefrom', 'volumeto'], axis=1, inplace=True)
df = df.merge(xlm, how='outer', on='date')
print(df.shape)
df.head(3)

(1131, 15)


,date,open,high,low,close,volumefrom,volumeto,vix,dxy,effr,btc,xrp,eos,ltc,xlm
0,2015-08-06,0.6747,3.0,0.6747,3.0,123.93,371.79,13.77,NaN,0.14,277.89,0.008013,NaN,4.10,NaN
1,2015-08-07,3.0000,3.0,0.1500,1.2,2119.43,1438.16,13.39,97.56,0.14,258.60,0.009000,NaN,3.77,NaN
2,2015-08-08,1.2000,1.2,1.2000,1.2,0.00,0.00,13.39,97.56,0.14,263.87,0.008700,NaN,3.87,NaN


In [125]:
# monero
xmr_raw = pd.read_csv('xmr_raw.csv')
xmr = xmr_raw[['timestamp', 'open', 'high', 'low', 'close', 'volumefrom', 'volumeto']].copy()
xmr['timestamp'] = pd.to_datetime(xmr['timestamp'])
xmr['timestamp'] = xmr['timestamp'].dt.date
xmr.rename(columns = {'timestamp':'date', 'close':'xmr'}, inplace=True)
xmr['date'] = pd.to_datetime(xmr['date'], format='%Y-%m-%d')
xmr = xmr.loc[(xmr['date'] >= start_date) & (xmr['date'] <= end_date)]
xmr.sort_values(by='date', inplace=True)
xmr.reset_index(drop=True, inplace=True)
xmr.drop(['high', 'low', 'open', 'volumefrom', 'volumeto'], axis=1, inplace=True)
df = df.merge(xmr, how='outer', on='date')
print(df.shape)
df.head(3)

(1131, 16)


,date,open,high,low,close,volumefrom,volumeto,vix,dxy,effr,btc,xrp,eos,ltc,xlm,xmr
0,2015-08-06,0.6747,3.0,0.6747,3.0,123.93,371.79,13.77,NaN,0.14,277.89,0.008013,NaN,4.10,NaN,0.75
1,2015-08-07,3.0000,3.0,0.1500,1.2,2119.43,1438.16,13.39,97.56,0.14,258.60,0.009000,NaN,3.77,NaN,0.75
2,2015-08-08,1.2000,1.2,1.2000,1.2,0.00,0.00,13.39,97.56,0.14,263.87,0.008700,NaN,3.87,NaN,0.75


In [129]:
df.to_pickle('crypto_df.pkl')

In [127]:
df.tail()

,date,open,high,low,close,volumefrom,volumeto,vix,dxy,effr,btc,xrp,eos,ltc,xlm,xmr
1126,2018-09-05,228.27,232.78,211.18,229.52,1217780.00,2.736424e+08,13.91,89.72,1.91,6515.42,0.3014,5.23,57.49,0.2079,117.21
1127,2018-09-06,229.56,234.15,213.83,215.14,712406.09,1.577458e+08,14.65,89.72,1.91,6411.78,0.2868,5.06,55.55,0.2065,110.87
1128,2018-09-07,215.14,220.02,190.08,196.77,866674.05,1.771377e+08,14.88,89.72,1.91,6200.16,0.2769,4.74,53.14,0.1943,103.81
1129,2018-09-08,196.77,208.38,185.01,195.99,803730.43,1.580799e+08,14.88,89.72,1.91,6249.07,0.2759,4.94,54.72,0.1912,105.07
1130,2018-09-09,195.99,202.59,186.92,196.98,649556.50,1.268624e+08,14.88,89.72,1.91,6319.76,0.2662,5.03,53.86,0.1896,105.56


In [128]:
print(df.xlm.isnull().sum())

529
